In [ ]:
singularity pull docker://nipy/heudiconv:latest

In [ ]:
# Get scan information and a heuristic template

singularity run \
-H ~/.singularity \
-B /Shared/oleary/:/hpc \
/Users/tientong/heudiconv-latest.simg \
-d /hpc/functional/UICL/dicomdata/{subject}/{session}/*/*.dcm \
-s 5002 \
-ss 64157414 \
-f convertall \
-c none \
-o /hpc/functional/UICL/BIDS

# everything will be store in /hpc/functional/UICL/BIDS/.heudiconv

Edit the heuristic file, saved to `heudiconv/convert.py`

In [ ]:
# run heudiconv for single subject

singularity run \
-H ~/.singularity \
-B /Shared/oleary/:/hpc \
/Users/tientong/heudiconv-latest.simg \
-d /hpc/functional/UICL/dicomdata/{subject}/{session}/*/64157414*.dcm \
-s 5002 \
-ss 64157414 \
-f /hpc/functional/UICL/BIDS/code/heudiconv/convert.py `# place where I saved the edited heuristic converter` \
-c dcm2niix -b `# converter I want to use (into bids format)` \
-o /hpc/functional/UICL/BIDS

In [ ]:
singularity run \
-H ~/.singularity \
-B /Shared/oleary/:/hpc \
/Users/tientong/heudiconv-latest.simg \
-d /hpc/functional/UICL/dicomdata/{subject}/{session}/*/*.dcm \
-s 3188 \
-ss 60898816 \
-f /hpc/functional/UICL/BIDS/.heudiconv/3188/ses-60898816/info/convert.py `# place where I saved the edited heuristic converter` \
-c dcm2niix -b `# converter I want to use (into bids format)` \
-o /hpc/functional/UICL/BIDS/rawdata

# Time 2  = 119 subjects

First get the list of subject id and session number

`/Shared/oleary/functional/UICL/BIDS/code/time2/time2_id.txt` = list of all subjects had "gender time 2" data in the SPSS file

Then, use the code below to get session id (both time 1 and time 2) for those subjects (who had gender time 2 data) -- echo to a text file named `time1time2_id.txt`

In [ ]:
%%bash
for sub in $(cat /Shared/oleary/functional/UICL/BIDS/code/time2/time2_id.txt | tr "\n" " ") ; do
    for i in /Shared/oleary/functional/UICL/dicomdata/$sub/[0-9]* ; do 
        ses=(`echo $i | awk '{gsub("/"," "); print $7}'`)
        echo ${sub}_${ses} >> time1time2_id.txt
    done
done

### Issue 1: Find the list of sub and ses for Time 2 data

Run this:

`/Shared/oleary/functional/UICL/BIDS/code/time2/gett2session.Rmd`

to get the session numbers of time 2, the list printed in this txt file 

`/Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt`


### Issue 2: For some subjects, ses included both corrected and uncorrected dicoms

this is post-processing done automatically. E.g., 3011 

`/Shared/oleary/functional/UICL/dicomdata/3011/63029716/T2_013/`

and

`/Shared/oleary/functional/UICL/dicomdata/3011/63029716/T2_1300/`

have to get rid of the ones with the bigger number (uncorrected).

First, get a list of all scans from all 119 people

In [ ]:
%%bash

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))

for n in `seq -s " " 0 1 118` ; do

    ls -d /Shared/oleary/functional/UICL/dicomdata/${sub[$n]}/${ses[$n]}/*/ >> /oleary/functional/UICL/BIDS/code/time2/allscanslist.txt

done

Then, run `/oleary/functional/UICL/BIDS/code/time2/move_extrascans.R` to move extra scans to `/Shared/oleary/functional/UICL/dicomdata/extra`.

# Back to Heudiconv

Create the template script below, which can be used to generate subject specific script

In [ ]:
%%bash

#!/bin/bash

subject_log=/Shared/oleary/functional/UICL/BIDS/code/time2/heudiconvT2_log/SUBJECT_SESSION.txt

echo '#--------------------------------------------------------------------------------' >> ${subject_log}

singularity run \
-H /Users/tientong/.singularity \
-B /Shared/oleary/:/hpc \
/Users/tientong/heudiconv-latest.simg \
-d /Shared/oleary/functional/UICL/dicomdata/{subject}/{session}/*/*.dcm \
-s SUBJECT \
-ss SESSION \
-f /Shared/oleary/functional/UICL/BIDS/code/heudiconv/convert.py \
-c dcm2niix -b \
-o /Shared/oleary/functional/UICL/BIDS/time2 2>&1 | tee ${subject_log}

date +"end_time: %Y-%m-%d_%H-%M-%S" >> ${subject_log}
echo '' >> ${subject_log}


use `sed` to replace placeholders with actual subject id and sessions to create subject specific script

In [ ]:
%%bash

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))

for n in `seq -s " " 0 1 118` ; do
    sed -e "s|SUBJECT|${sub[$n]}|" -e "s|SESSION|${ses[$n]}|" heudiconv_TEMPLATE.sh > heudiconv_sub-${sub[$n]}_ses-${ses[$n]}.sh
done

run multiple scripts:

`ls heudiconv_sub* | xargs -i{} -P5 sh {}`

### Issue 3: data from multiple scanners - GE and siemens

have to make a different heuristic

`/Shared/oleary/functional/UICL/BIDS/code/heudiconv/convert2.py`

In [ ]:
%%bash

# job submission on Argon cluster for all subject-specific heudiconv

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_group2.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_group2.txt))

for n in `seq -s " " 0 1 97` ; do
    sed -e "s|SUBJECT|${sub[$n]}|" -e "s|SESSION|${ses[$n]}|" heudiconv_TEMPLATE.sh > heudiconv_sub-${sub[$n]}_ses-${ses[$n]}.sh
    qsub heudiconv_sub-${sub[$n]}_ses-${ses[$n]}.sh
done

## For resting state time 2

has to use /Shared/oleary/functional/UICL/BIDS/code/heudiconv/convert3.py for Siemens scans

In [114]:
# find subjects scanned with siemens

import pandas as pd

time2 = pd.read_csv('/Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt', 
                    sep=" ", header = None)
ge = pd.read_csv('/Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_group2.txt', 
                 sep=" ", header = None)

siemens = pd.concat([time2,ge]).drop_duplicates(keep=False)

fullsample_sub = pd.read_csv('/Shared/oleary/functional/UICL/BIDS/subject_list/allsub.txt', header = None)
fullsample_ses = pd.read_csv('/Shared/oleary/functional/UICL/BIDS/subject_list/session1.txt', header = None)
fullsample = pd.merge(fullsample_sub, fullsample_ses, left_index=True, right_index=True)
fullsample.to_csv('/Shared/oleary/functional/UICL/BIDS/subject_list/allsub_session1.txt', 
               sep='\t', index = False, header = False)

time1only = fullsample[-fullsample['0_x'].isin(time2[1])].reset_index(drop=True)
time1only.to_csv('/Shared/oleary/functional/UICL/BIDS/subject_list/time1only.txt', 
               sep='\t', index = False, header = False)


siemens.columns = ['id_time2','id','time2']
siemens.reset_index(inplace=True, drop=True)
siemenstime1 = fullsample[fullsample['0_x'].isin(siemens['id'])]['0_y']
siemenstime1.reset_index(inplace=True, drop=True)
siemens['time1'] = siemenstime1
siemens.to_csv('/Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_siemens.txt', 
               sep='\t', index = False, header = False)

ge.columns = ['id_time2','id','time2']
getime1 = fullsample[fullsample['0_x'].isin(ge['id'])]['0_y']
getime1.reset_index(inplace=True, drop=True)
ge['time1'] = getime1
ge.to_csv('/Shared/oleary/functional/UICL/BIDS/code/time2/ge_time1time2.txt', 
               sep='\t', index = False, header = False)

time2 = pd.merge(ge, siemens,how='outer')
time2.to_csv('/Shared/oleary/functional/UICL/BIDS/code/time2/alltime2.txt', 
               sep='\t', index = False, header = False)

run the `get_slice_time.py` script to get the slice timing information for GE scanner.

```
usage: python /path/to/get_slice_time.py /path/to/dicom/pattern* /out/path

python /Shared/oleary/functional/UICL/BIDS/code/get_slice_time.py "/Shared/oleary/functional/UICL/dicomdata/3040/63910816/FMRI_004/63910816_004_*" "/Shared/oleary/functional/UICL/BIDS/code/time2/"

```

In [107]:
%%bash
subid=($(cut -f4 /Shared/oleary/functional/UICL/BIDS/code/time2/ge_time1time2.txt))
for i in ${!subid[@]}; do echo ${subid[$i]}; done

60913814
64069714
64182514
64184914
64066514
64559714
64444714
64558514
64459014
64744514
64977014
64645914
64572314
64747014
65047514
64975114
64990514
65147714
65060914
65076014
60223115
65062714
65248914
60093215
60527515
60498815
60628415
60483615
60512015
60597115
60595615
60700715
60540315
60585615
60926915
61045415
60801315
60712915
60743615
60900715
60930815
61027015
61348315
61231215
61290115
61017415
61333815
61230315
61491515
61346915
61388015
61535415
61705915
61895515
61835415
61822415
61907115
63764115
63937815
63853815
63739515
63710315
64040415
63825815
64012615
64128115
64153315
64154215
63868015
64069515
64356315
64573115
64457015
64917015
64632215
64470115
65050315
64859015
64948715
64861115
65061515
60511116
60585716
60667716
60683216
60815816
60700916
60900316
60898816
61101116
61636716
61680116
61835516
61722516
61823616
61837616
61924316
61936216
